In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel, LdaModel
import matplotlib.pyplot as plt
import logging
import pandas as pd # Assuming negative_reviews is a Pandas DataFrame/Series

# --- Configuration du Logging ---
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# ==============================================================================
# DÉFINITION DE LA FONCTION DE COHÉRENCE (Version adaptée pour Gensim)
# ==============================================================================
def get_Cv_gensim(model, texts, dictionary, coherence_type='c_v'):
    """
    Calcule et renvoie le score de cohérence pour un modèle LDA Gensim.

    Args:
        model (gensim.models.ldamodel.LdaModel): Le modèle LDA entraîné.
        texts (list of list of str): La liste des textes tokenisés.
        dictionary (gensim.corpora.Dictionary): Le dictionnaire Gensim.
        coherence_type (str, optional): Le type de mesure de cohérence ('c_v', 'u_mass', etc.).
                                        Défaut à 'c_v'.

    Returns:
        float or None: Le score de cohérence, ou None en cas d'erreur.
    """
    if not texts or not dictionary or not model:
        logging.error("Modèle, Textes ou Dictionnaire manquant pour le calcul de cohérence.")
        return None
    try:
        coherence_model = CoherenceModel(
            model=model,
            texts=texts,
            dictionary=dictionary,
            coherence=coherence_type
        )
        coherence = coherence_model.get_coherence()
        return coherence
    except Exception as e:
        logging.error(f"Erreur lors du calcul de la cohérence ({coherence_type}) dans get_Cv_gensim: {e}")
        return None
# ==============================================================================


# ==============================================================================
# DÉFINITION DE LA FONCTION POUR TROUVER LE NOMBRE OPTIMAL DE TOPICS
# ==============================================================================
def find_optimal_number_of_topics(dictionary, corpus, texts, limit, start=2, step=3, random_state=100, passes=10, iterations=50, chunksize=100, coherence_type='c_v'):
    """
    Calcule les scores de cohérence pour différents nombres de topics et retourne
    le nombre de topics optimal ainsi que les modèles et scores associés.

    Args:
        dictionary (gensim.corpora.Dictionary): Le dictionnaire Gensim.
        corpus (list): Le corpus au format BoW Gensim.
        texts (list of list of str): La liste des textes prétraités (liste de listes de tokens).
        limit (int): Le nombre maximum de topics à tester (exclusif).
        start (int, optional): Le nombre minimum de topics à tester. Défaut à 2.
        step (int, optional): L'incrément entre les nombres de topics testés. Défaut à 3.
        random_state (int, optional): Graine pour la reproductibilité. Défaut à 100.
        passes (int, optional): Nombre de passes LDA. Défaut à 10.
        iterations (int, optional): Nombre d'itérations LDA. Défaut à 50.
        chunksize (int, optional): Taille des chunks LDA. Défaut à 100.
        coherence_type (str, optional): Type de cohérence à utiliser ('c_v'). Défaut à 'c_v'.


    Returns:
        tuple: (optimal_num_topics, coherence_scores_dict)
               - optimal_num_topics (int or None): Le nombre optimal de topics trouvé, ou None si échec.
               - coherence_scores_dict (dict): Dictionnaire {num_topics: coherence_score}.
    """
    coherence_values = []
    model_list = [] # Vous pourriez vouloir retourner cette liste si vous voulez les modèles
    topic_numbers = list(range(start, limit, step))

    if not topic_numbers:
        logging.warning(f"La plage de topics spécifiée (start={start}, limit={limit}, step={step}) est vide.")
        return None, {}

    logging.info(f"Début du calcul de cohérence ({coherence_type}) pour {len(topic_numbers)} modèles (topics de {start} à {limit-1} par pas de {step})...")

    for num_topics in topic_numbers:
        logging.info(f"Entraînement du modèle LDA pour {num_topics} topics...")
        try:
            # Entraînement du modèle LDA Gensim
            model = LdaModel( # Utilisation directe de LdaModel après import
                corpus=corpus,
                id2word=dictionary,
                num_topics=num_topics,
                random_state=random_state,
                update_every=1,
                chunksize=chunksize,
                passes=passes,
                iterations=iterations,
                alpha='auto',
                eta='auto',
                per_word_topics=True # Nécessaire pour certaines visualisations, peut être False
            )
            model_list.append(model) # Stocker le modèle si besoin

            # Calcul du score de cohérence
            logging.debug(f"Appel de get_Cv_gensim pour {num_topics} topics...")
            coherence = get_Cv_gensim(model=model, texts=texts, dictionary=dictionary, coherence_type=coherence_type)

            if coherence is not None:
                coherence_values.append(coherence)
                logging.info(f"Nombre de Topics = {num_topics} -> Score de Cohérence ({coherence_type}) = {coherence:.4f}")
            else:
                logging.warning(f"get_Cv_gensim a retourné None pour {num_topics} topics. Ce point sera ignoré.")
                coherence_values.append(None) # Ajouter None pour garder l'alignement avec topic_numbers

        except Exception as e:
            logging.error(f"Erreur lors de l'entraînement ou du calcul de cohérence pour {num_topics} topics: {e}")
            coherence_values.append(None) # Ajouter None en cas d'erreur

    # Filtrer les éventuels échecs (None) avant de chercher le max
    # Créer une liste de tuples (nombre_topic, score) uniquement pour les scores valides
    valid_scores_tuples = []
    for i, score in enumerate(coherence_values):
        if score is not None:
            valid_scores_tuples.append((topic_numbers[i], score))

    if not valid_scores_tuples:
        logging.warning(f"Aucun score de cohérence ({coherence_type}) n'a pu être calculé avec succès.")
        return None, {} # Retourner un dictionnaire vide pour les scores

    # Trouver le nombre de topics avec le score maximal
    optimal_num_topics, max_coherence = max(valid_scores_tuples, key=lambda item: item[1])
    # Créer le dictionnaire final des scores valides
    coherence_scores_dict = dict(valid_scores_tuples)

    logging.info(f"Calcul terminé. Nombre optimal de topics trouvé : {optimal_num_topics} (Score {coherence_type} = {max_coherence:.4f})")

    # Retourner le nombre optimal et le dictionnaire des scores
    return optimal_num_topics, coherence_scores_dict
# ==============================================================================


# ==============================================================================
# CHARGEMENT ET PRÉPARATION DES DONNÉES (EXEMPLE)
# ==============================================================================
# !!! Remplacez ceci par votre propre chargement et prétraitement de données !!!
# Exemple: Supposons que vous ayez un DataFrame pandas 'df' avec une colonne 'text'
# contenant vos textes déjà nettoyés et lemmatisés.
data = {
    'text': [
        "le service client est déplorable attente interminable",
        "produit reçu cassé très déçu qualité médiocre",
        "application bug souvent impossible utiliser correctement",
        "livraison retardée sans aucune information utile",
        "ne correspond pas description retour demandé",
        "mauvaise expérience utilisateur interface compliquée",
        "le support technique répond jamais aux emails",
        "qualité prix pas justifiée trop cher pour ce que est",
        "publicité mensongère sur les fonctionnalités réelles",
        "annulation commande sans explication service nul"
    ]
}
negative_reviews = pd.DataFrame(data) # Crée un DataFrame exemple

# --- Tokenisation ---
# Utiliser text.split() est simple, mais un tokenizer plus avancé (NLTK, spaCy)
# est souvent préférable pour mieux gérer la ponctuation, etc.
# Assurez-vous que cette étape correspond à votre prétraitement réel.
try:
    # Assurez-vous que la colonne 'text' contient bien des chaînes de caractères
    if not all(isinstance(text, str) for text in negative_reviews["text"]):
         raise TypeError("La colonne 'text' doit contenir des chaînes de caractères.")

    tokenized_texts = [text.lower().split() for text in negative_reviews["text"]] # Ajout de .lower()
    logging.info(f"{len(tokenized_texts)} documents tokenisés.")

    # --- Création du Dictionnaire et du Corpus Gensim ---
    dictionary = corpora.Dictionary(tokenized_texts)
    # Optionnel: Filtrer les extrêmes (mots très rares ou très fréquents)
    # dictionary.filter_extremes(no_below=5, no_above=0.5)
    logging.info(f"Dictionnaire créé avec {len(dictionary)} mots uniques.")

    corpus = [dictionary.doc2bow(text) for text in tokenized_texts]
    logging.info(f"Corpus créé avec {len(corpus)} documents.")

    # Vérification rapide que le corpus n'est pas vide
    if not corpus or not any(corpus):
         raise ValueError("Le corpus est vide ou tous les documents sont vides après tokenisation/filtrage.")

except KeyError:
    logging.error("La colonne 'text' n'existe pas dans les données fournies (negative_reviews).")
    exit() # Arrêter si les données de base manquent
except TypeError as te:
     logging.error(f"Erreur de type lors de la tokenisation : {te}. Vérifiez le contenu de 'negative_reviews['text']'.")
     exit()
except ValueError as ve:
     logging.error(f"Erreur de valeur lors de la création du corpus : {ve}.")
     exit()
except Exception as e:
    logging.error(f"Une erreur inattendue est survenue lors de la préparation des données : {e}")
    exit()
# ==============================================================================


# ==============================================================================
# RECHERCHE DU NOMBRE OPTIMAL DE TOPICS
# ==============================================================================
# --- Paramètres pour la recherche ---
start_topics = 2     # Nombre minimum de topics à tester
limit_topics = 11    # Nombre maximum de topics à tester + 1 (range s'arrête avant)
step_topics = 1      # Incrément entre les nombres de topics
lda_passes = 10      # Nombre de passes pour l'entraînement LDA pendant la recherche
lda_iterations = 50  # Nombre d'itérations pour l'entraînement LDA

# --- Appel de la fonction ---
optimal_topics, coherence_scores = find_optimal_number_of_topics(
    dictionary=dictionary,
    corpus=corpus,
    texts=tokenized_texts, # Utiliser les textes tokenisés
    limit=limit_topics,
    start=start_topics,
    step=step_topics,
    passes=lda_passes,
    iterations=lda_iterations,
    coherence_type='c_v' # Spécifier 'c_v' explicitement si désiré
)
# ==============================================================================


# ==============================================================================
# AFFICHAGE DES RÉSULTATS ET VISUALISATION
# ==============================================================================
if optimal_topics is not None:
    print(f"\n--- Résultats de la Recherche ---")
    print(f"Le nombre optimal de topics (basé sur la cohérence 'c_v') est : {optimal_topics}")
    print("Scores de cohérence ('c_v') par nombre de topics :")
    # Trier le dictionnaire par nombre de topics pour un affichage ordonné
    sorted_scores = sorted(coherence_scores.items())
    for topics, score in sorted_scores:
        print(f"  {topics} topics : {score:.4f}")

    # --- Visualisation des scores de cohérence ---
    try:
        x = list(coherence_scores.keys())
        y = list(coherence_scores.values())

        plt.figure(figsize=(10, 6))
        plt.plot(x, y, marker='o', linestyle='-', color='b')
        plt.xlabel("Nombre de Topics")
        plt.ylabel("Score de Cohérence (c_v)")
        plt.title("Évolution du Score de Cohérence (c_v) vs. Nombre de Topics")
        plt.xticks(range(start_topics, limit_topics, step_topics)) # Assurer des marques claires
        plt.grid(True, linestyle='--', alpha=0.6)

        # Marquer le point optimal
        if optimal_topics in coherence_scores:
             plt.plot(optimal_topics, coherence_scores[optimal_topics], marker='X', color='red', markersize=12, linestyle='None', label=f'Optimal ({optimal_topics} topics)\nScore={coherence_scores[optimal_topics]:.4f}')
             plt.legend(loc='best')

        plt.tight_layout() # Ajuster la mise en page
        plt.show()

    except ImportError:
        logging.warning("Matplotlib non trouvé. Installez-le (`pip install matplotlib`) pour visualiser les scores.")
    except Exception as e:
        logging.error(f"Erreur lors de la création du graphique : {e}")

else:
    print("\nImpossible de déterminer le nombre optimal de topics avec les paramètres fournis.")
# ==============================================================================


# ==============================================================================
# OPTIONNEL : ENTRAÎNEMENT DU MODÈLE LDA FINAL AVEC LE NOMBRE OPTIMAL DE TOPICS
# ==============================================================================
if optimal_topics is not None:
    print(f"\n--- Entraînement du Modèle LDA Final ---")
    print(f"Entraînement du modèle LDA final avec {optimal_topics} topics...")

    # Vous pouvez augmenter les passes/itérations pour le modèle final
    final_passes = 20
    final_iterations = 100

    try:
        final_lda_model = LdaModel(
            corpus=corpus,
            id2word=dictionary,
            num_topics=optimal_topics, # Utilisation de la variable optimale
            random_state=100,          # Garder la même graine pour la reproductibilité
            update_every=1,
            chunksize=100,             # Garder les mêmes hyperparamètres que lors de la recherche
            passes=final_passes,       # Ou utiliser des valeurs potentiellement plus élevées
            iterations=final_iterations,
            alpha='auto',
            eta='auto',
            per_word_topics=True       # Utile pour certaines analyses ultérieures
        )
        print("Modèle final entraîné avec succès.")

        # --- Afficher les topics du modèle final (Exemple) ---
        print(f"\nTopics trouvés par le modèle final ({optimal_topics} topics):")
        # Utiliser print_topics pour une meilleure lisibilité
        topics = final_lda_model.print_topics(num_words=10) # Afficher les 10 mots principaux par topic
        for topic_num, topic_words in topics:
            print(f"Topic {topic_num}: {topic_words}")

        # Vous pouvez maintenant utiliser 'final_lda_model' pour d'autres tâches
        # (ex: classification, analyse de documents spécifiques, etc.)

    except Exception as e:
        logging.error(f"Erreur lors de l'entraînement du modèle LDA final : {e}")

# ==============================================================================

print("\nScript terminé.")
